In [ ]:
%%html
<style>
div.TOC {    
    background-color: #fcfcfc;
    border-color: #dFdFdF;
    border-left: 5px solid #dFdFdF;
    padding: 0.5em;
    }
div.TOC a {
  color: grey;
  text-decoration: none;
  font-weight: 500;
}
.ToTOC a {
    color:#AAAAAA;
    font-size:18px;
    text-decoration: none;
}
 </style>

# Lab: Systems of equations:
<a name="TableOfContents"></a> 

### Table of contents 
<div class="TOC">
<ol>
    <li>[Warmup: Autonomous Equations](#warmup)</li>
    <ul style="list-style-type:none;margin-top:0">
        <li>[A little bit about loops](#loops)</li>
    </ul>
    <li>[Systems of Differential Equations](#systemsofdiffeqs)</li>
    <ul style="list-style-type:none;margin-top:0">
        <li>[A little bit about loops](#loops)</li>
        <li>[Applications](#apps2)</li>
        <ul style="list-style-type:none;margin-top:0">
            <li>[2.1 Coupled Linear Systems](#CLS)</li>
            <li>[2.2 Coupled Nonlinear Systems](#CLNS)</li>
            <li>[2.3 Comparing To Data](#CTD)</li>
            <li>[2.4 Coupled Non-smooth Systems](#CNSS)</li>
        </ul>
    </ul>
    <li>[Case Study: Cholera Outbreak](#CASE1)</li>
    <li>[Case Study: Michaelis–Menten kinetics](#CASE2)</li>
</ol>
    Case studies were taken from *Mathematical Modelling with Case Studies - Third Edition*, B. Barnes and G. R. Fulford, CRC Press, 2015
</div>


## 1. Warmup: Autonomous Equations <a name="warmup"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

First, lets import all of our libraries: numpy for our numerical toolkit, odeint to solve ODE's and the pyplot section of matplotlib to graph our data.

In [ ]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import numpy as np

Lets warm up by solving numerically the differential equation from the homework:

$$
\frac{dy}{dt} = \sin(y)
$$

We need to define a model, using `np.sin()` from the numpy library, and solve it using `odeint(model, y0, t)`, for initial condition $y(0) = .1$. Dont forget to define your time range using `np.linspace`, your time range should be between 0 and 20 approximately. 

In [ ]:
def model(y,t):
    dydt = np.sin(y)
    return dydt

t = np.linspace(0,20)
y0 = .1

y_sol = odeint(model, y0, t)


plt.plot(t,y_sol)

### A little bit about loops: <a name="loops"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

Lets say we want to plot 10 different initial conditions on the same plot to get a better feel for the space. One way to explore the solutions space would be to manually code in 10 different `y0` values, manually run odeint on each, and finally plot them all together. However, computers a very good at doing thing over and over, can Python do it for us?

A **loop** is a piece of code that is run over and over again, possibly with changing variables. Python has a unique way of defining loops in terms of **lists**. Let's show a quick example, take a look at the code below and then run it.

In [ ]:
a_list = [1,3,7,12,-4]

for var in a_list:      # Note,`var' is just a name, it could be anything
    print(var)
    print(2*var)
    print(var**2)
    print()

In the code above, we define a list called `a_list` and put some words in it. Note, that any text must be surrounded by `''` marks. The **for** loop sets a variable called `var` to each of the elements of the list and runs the code below it on `var`, first it just prints `var`, then it multiplies by 2 and prints it, then squares it and prints it. 

In Python, a for loop just sets a variable to each element of a list and then runs some code. Try using a for loop to solve the ODE $y' = sin(y)$ and plot the solutions for $y_0 = 1,\,2,\,3,\,4,\,5,\,6,\,7,\,8,\,9,\,10$. 

In [ ]:
initial_values = [0,1,2,3,4,5,6,7,8,9,10, 2*np.pi, 2*(3.14159)]

for y0 in initial_values:
    y_sol = odeint(model, y0, t)
    plt.plot(t,y_sol)

Note: What happens if you start on the unstable equilibrium solution $2\pi$? Try it using `2*np.pi` and `2*(3.14159)`. Why do you think there's a difference?

## 2. Systems of differential equations <a name="systemsofdiffeqs"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

The `odeint` solver can also handle coupled systems of differential equations, the model function just needs to output a *vector* $\left[\frac{dx}{dt},\frac{dy}{dt}\right]$ instead of a single *value* like $\frac{dy}{dt}$. For example, consider the coupled equations

\begin{align}
\frac{dx}{dt} &= y(t)\,,\hspace{4em} &x(0)&=0\,,
\\
\frac{dy}{dt} &= -x(t)\,, &y(0)&=1\,.
\end{align}

We can solve this system of equations using `odeint` by promoting `y` to a vector `Y = [x,y]`, and making our function `model(Y,t)` a function of `Y` that returns a vector `dYdt = [dxdt,dydt]`.

In [ ]:
def model(Y,t):
    x,y = Y
    dYdt = [y,-x]
    return dYdt

Take a second and run this function on different inputs.

In [ ]:
print(model([1,2],0))

Now, to fit we just set the starting values for `x` and `y` by specifying `Y0 = [0,1]` and run the estimator.

In [ ]:
Y0 = [0,1]
t = np.linspace(0,10,100)

sol = odeint(model,Y0,t)

print(sol)

The first column of `sol` is the list of `x` values associated to each `t` value and the second column is the list of `y` values. We can plot these on the graph. 

How do we get a column of just the `x` values or just the `y` values? Python uses matrix like notation to access elements of a list, except that **the first column and first row is always 0 not 1**.

For example, `sol[4,0]` is the 5'th row of column 0 and `sol[4,1]` is the 5'th row of column 1: 

In [ ]:
print(sol[4,0])
print(sol[4,1])

To get *all* of the rows in column 1, we use the slice delimitator `:`. The slice delimitator helps us select subsets of a list, for examples `sol[3:7,0]` returns rows 3 to (7-1) of the column 0, that is rows 3, 4, 5, 6:

In [ ]:
print(sol[3:7,0])

To get a full column, we just use the slice operator alone. For example `sol[:,0]` returns the whole of column 0, just like `sol[4,:]` returns all of row 4. 

In [ ]:
xsol = sol[:,0]
ysol = sol[:,1]

We can now plot our solutions for $x$ and $y$:

In [ ]:
## 
plt.plot(t,xsol)
plt.plot(t,ysol)

*or* think about them as tracing out a path $(x(t), y(t)) \in \mathbb{R}^2$ by plotting them against each other. 

In [ ]:
plt.plot(xsol,ysol)

Note, we could also just plot the columns of the array of solutions:

In [ ]:
plt.plot(sol[:,0],sol[:,1])

## Applications: <a name="apps2"></a>

### 2.1 Coupled Linear Systems <a name="CLS"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

Use a model with vector input `Y` and output `[dxdt,dydt]` to solve the initial value problem 

\begin{align}
\frac{dx}{dt} &= .7x(t) + y(t)\,,\hspace{4em} &x(0)&=0\,,
\\
\frac{dy}{dt} &= -x(t)\,, &y(0)&=1\,.
\end{align}

Set up a `model` for this system and solve it for $t\in [0,20]$ using `odeint` and plot $x$ vs $t$, $y$ vs $t$ and $x$ vs $y$..

### 2.2 Coupled Nonlinear Systems <a name="CNLS"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

Use a model with vector input `Y` and output `[dxdt,dydt]` to solve the initial value problem 

\begin{align}
\frac{dx}{dt} &= x(1-x) - 5\frac{xy}{1+x} \,,\hspace{4em} &x(0)&=1\,,
\\
\frac{dy}{dt} &= 5\frac{xy}{1+x} - y \,, &y(0)&=1\,.
\end{align}

Set up a `model` for this system and solve it for $t\in [0,100]$ using `odeint` and plot $x$ vs $t$, $y$ vs $t$ and $x$ vs $y$.

### 2.3 Comparing To Data: <a name="CTD"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

In ["Anonymous (1978). Influenza in a boarding school"](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1603269/pdf/brmedj00115-0064.pdf) the number of cases per day of an influenza infection in a English boarding school was measured. This information is summarized in a database by the [R Epidemics Consortium (RECON)](http://www.repidemicsconsortium.org/outbreaks/index.html), the details of which can be found [here](http://www.repidemicsconsortium.org/outbreaks/reference/influenza_england_1978_school.html). Below, we put this information in an object called a **data frame**. A data frame is a table with headings, so that you can access the entries in a more human readable way.  



In [ ]:
d = {'day': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
     'in_bed': [3, 7, 22, 78, 233, 300, 256, 233, 189, 128, 72, 33, 11, 6]}

influenza = pd.DataFrame(d)

The columns of the dataframe can be access by name by using `influenza['day']`

In [ ]:
print(influenza['day'])
print(influenza['in_bed'])

We can plot the number of sick children from the 'influenza' dataset using the same plot function at before, only this time we may want to change from a line plot to a scatter plot. We do this by adding a flag `'o'` after the `x` and `y` data:

In [ ]:
plt.plot(influenza['day'],influenza['in_bed'],'o')

Lets return to our SIR epidemic model. Since the recovered patients don't feed back into the system we will take a moment to analyze only the susceptible and infected patients:

\begin{align}
\frac{dS}{dt} &= -\beta S I\,,
\\
\frac{dI}{dt} &= \beta S I- \gamma I\,,
\end{align}

for $t\in[0,20]$. Lets fix the transmission coefficiant $\beta = 2.18\times 10^{-3}$ and the recovery rate $\gamma = 0.44$, $I(0) = 0$ and $S(0) = 100, 500, 800$. Setup and solve a model for this differential equation and plot the results.

In the study above, there were suseptable 762 students in the boarding school. Plot the solved model and the actual number of infections on the same axis. 

### 2.4 Coupled Non-smooth Systems <a name="CNSS"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

As before, let $u(t)$ be the step function $u(t) = \begin{cases}0& \text{for }t<10
\\
2& \text{for }t\geq 10
\end{cases}\,.
$ Solve the initial value problem 

\begin{align}
\frac{dx}{dt} &= -x(t) + u(t)\,,\hspace{4em} &x(0)&=1\,,
\\
\frac{dy}{dt} &= 6x(t)-y(t)\,, &y(0)&=0\,,
\end{align}

for $t\in[0,20]$.

## Case Study: Cholera Outbreak <a name="CASE1"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

#### Background:

Based on [Endemic and Epidemic Dynamics of Cholera: The Role of the Aquatic Reservoir](https://www.researchgate.net/publication/12123535_Endemic_and_Epidemic_Dynamics_of_Cholera_The_Role_of_the_Aquatic_Reservoir) and the textbook.

Cholera is a water born disease that is particularly dangerous in areas where sanitation is inadequate and sewage can find its way into the drinking water supply. Modeling can provide an understanding of circumstances under which an outbreak can occur. Here we formulate a model that include interacting susceptible and infectious populations. However, what is different from the usual case is twofold: first, we include transmission from the environment and second we imagine tracking the disease over a long period of time and so we include birth and death rates. 

#### Governing Equations:
The variables needed to describe the prevalence of cholera in the population are $S(t)$, susceptible, and $I(t)$, infective, where $t$ is time. Another important variable is the concentration of cholera bacteria in the water supply. We will use $B(t)$ to represent bacterial concentration. This will change in time as more bacteria enter the water supply through ongoing sewage contaminations, which then increases with an increasing number of infectives shedding cholera bacteria. 


|$S$| Number of susceptible 
|$I$| Number of infected 
|$B$| Concentration of cells in water (cells/ml)


#### State Variables:

|Parameter| Description |
|---|---|
|$S$| Number of susceptible |
|$I$| Number of infected |
|$B$| Concentration of cells in water (cells/ml)|

#### Parameters:

|Parameter| Description | Village 1 | Village 2 | Village 3|
|---|---|---|---|---|
|$H$| Total human population |10,000|10,000|10,000|
|$n$| Background birth and death rate (days$^{-1}$) |0.0001|0.0001|0.01|
|$a$| Rate of exposure to contaminated water (days$^{-1}$) |.5|1|1|
|$k$| Concentration of bacteria that leads to 50% infection rate (cells/ml) |$10^6$|$10^6$|$10^6$|
|$r$| Recovery rate (days$^{-1}$) |0.2|0.2|0.2|
|$n_b$| Growth/death rate of cholera bacteria in water (days$^{-1}$) |-.33|-.33|-.33|
|$e$| Rate of bacteria excretion per person $\left(\frac{\text{cells}}{\text{(ml)(days)(person)}}\right)$|10|10|10|

#### Equations:

We set up the following system of differential equations:

\begin{align}
\frac{dS}{dt} &= n(H-S) -\lambda(B)S\,,
\\
\frac{dI}{dt} &= \lambda(B)S - rI\,,
\\
\frac{dB}{dt} &= Bn_b + eI\,.
\end{align}

Here, $\lambda(B)$ is the **force of infection** and we assume that cholera is only contracted through contact with the environment and not through person-to-person contact. The force of infection is the probability per unit time of a susceptible being infected. While we could assume the probability is proportional to the bacterial concentration $B(t)$ times rate of exposure $a$, it is more realistic to assume it is linear for small $B(t)$ and then tends to 1 (100% of infection) as $B$ becomes large. We thus use a standard **hill function**

$$
\lambda(B) = ap(B) = a\frac{B}{k+B}\,.
$$

Substituting this in for $\lambda(B)$ we obtain the equations

\begin{align}
\frac{dS}{dt} &= n(H-S) -a\frac{B}{k+B}S
\\
\frac{dI}{dt} &= a\frac{B}{k+B}S - rI
\\
\frac{dB}{dt} &= Bn_b + eI
\end{align}

### Question 1:

In class, we derived the **critical town size** $S_C$ from the basic reproduction number and gave the formula

$$
S_C = -\frac{rkn_b}{e}
$$

What is the critical town size for each village? What do we expect this means for the chance of an outbreak?

* Village 1: $S_C = $
* Village 2: $S_C = $
* Village 3: $S_C = $

### Question 2: Village 1

Set up the model for this system of differential equations and run it for Village 1. For initial conditions, use $S(0) = H-10$, $I(0) = 10$ and $B(0) = 0$ to simulate an external infection over 300 days.

Plot $S(t)$, $I(t)$ and $B(t)$ on separate axes. How do you interpret your results?

### Question 3: Village 2

Set up the model for this system of differential equations and run it for Village 2. For initial conditions, use $S(0) = H-10$, $I(0) = 10$ and $B(0) = 0$ to simulate an external infection over 300 days.

Plot $S(t)$, $I(t)$ and $B(t)$ on **the same** axis and include the critical town size. How do you interpret your results? Add a legend to this axis to describe the different plots.

### Question 4: Village 3

Set up the model for this system of differential equations and run it for Village 2. For initial conditions, use $S(0) = H-10$, $I(0) = 10$ and $B(0) = 0$ to simulate an external infection but this time let the **time extend to 1000 days**. 

Plot $S(t)$, $I(t)$ and $B(t)$ on **the same** axis and include the critical town size. How do you interpret your results? Add a legend to this axis to describe the different plots.

## Case Study: Michaelis–Menten kinetics <a name="CASE2"></a><span class = "ToTOC"><a href="#TableOfContents" style="text-decoration:none">&#x2BA5;</a></span>

#### Background:

The following text is copied or paraphrased from

http://elte.prompt.hu/sites/default/files/tananyagok/IntroductionToPracticalBiochemistry/ch09s02.html
https://ocw.mit.edu/courses/physics/8-591j-systems-biology-fall-2004/readings/l2_syllabus.pdf

When the topic of enzyme kinetics first emerged, almost nothing was known about the physical nature of enzymes and the possible mechanisms of rate enhancement. Let us start with a thought experiment considering the dependence of the rate of a simple chemical reaction as a function of reactant concentration $S$. In this case, the rate of the $S\longrightarrow P$ reaction can be written as $V = dP/dt = kS$. In other words, the *rate of the reaction is linearly proportional to the concentration of the reactant $S$*. In principle, the rate could be increased to “infinity”—the only limit would be set by the solubility of $S$. 

Experimentally however this was not the case for two important reasons: First, the initial velocity of product formulation does not scale proportionally to the concentration of substrate. Second there is always a maximum initial velocity, that is the dependence of $dP/dt$ on $S$ is asymptotic as in the images below.

<img width=500 src = "http://elte.prompt.hu/sites/default/files/tananyagok/IntroductionToPracticalBiochemistry/images/1e780e74.jpg">

The first kinetic model that successfully explained this phenomenon was introduced by Leonor Michaelis and Maud Menten. Their presumption, which nowadays might seem trivial, was revolutionary in their time. They assumed that the enzyme $E$ directly interacts with the substrate $S$ in a stoichiometric manner, the interaction results in a well-defined intermediate complex $ES$, and the interaction leads to thermodynamic equilibrium $dES/dt = 0$. As a tribute to this first successful model, $ES$ has been named the Michaelis complex.

$$
E + S \underset{b}{\overset{a}\rightleftharpoons} ES \overset c\rightarrow P
$$

Note that there is both a forward and a back reaction between the enzyme $E$, the reactant $S$ and $ES$ but only a forward reaction between the intermediate complex $ES$ and the product $P$. It is usually assume that binding of enzyme and substrate to the $ES$ complex is almost instantiations, and much larger than the breaking of $ES$ complex back into enzyme and substrate or production of product. Mathematically, that means that $a\gg b\gg c$.

In the model above the reactant $S$ is turn into substrate $ES$ and then into product $P$, while the enzyme $E$ is released back into the system during $ES\longrightarrow P$. From the reaction model above we construct a system of four differential equations

\begin{align}
\frac{dS}{dt} = &\, -aE \cdot S + b ES
\\
\frac{dE}{dt} = &\, -aE \cdot S + (b+c) ES
\\
\frac{dES}{dt} = &\, aE \cdot S - (b+c) ES
\\
\frac{dP}{dt} = &\, cES
\end{align}

### Question 1:
Write and solve the differential equation for Michaelis–Menten kinetics for $a = 1000$, $b = 1$, $c=.05$, $S(0) = .001$, $E(0) = .0005$ and $ES(0) = P(0) = 0$ on three different time scales, $t \in [0,1]$, $t\in [0,10]$ and $t\in [0,100]$. Describe what's happens in each of these regimes. 

### Question 2.a:
The best models are those that make predictions. Michaelis and Menten wanted to show that the behavior expected by a enzyme catalyst model behaved substantially differently than a simple product to reactant model.

Increase $S(0)$ from .001 to .01 and plot the resulting model. Notice that model effectively becomes a *linear* exchange of product and reactant in the short term. 

### Question 2.b:

Contrasting the graph above to the exponential model $\frac{dP}{dt} = k S$ by creating a model for the system of equations governing $S\longrightarrow P$ and solving it. In this model use $S(0)=.01$ but choose $k$ so that the models are on approximately the same time scale. How are these models different?

### Question 3:

The linear behavior is caused by a so called quasi-equilibrium or pseudo-steady state in the $E$ and $ES$ quantities. The idea is as follows: If there is a fixed amount of $ES$ than the production of $P$ will be linear, *regardless of the amount of reactant $S$*, ie $\frac{dE}{dt} = \frac{dES}{dt} = 0$. 

Recall that the enzyme is never used up, so the total amount of $E$ plus $ES$ is equal to $E(0) = E_0$. Using $E = E_0-ES$, we can rewrite $\frac{dES}{dt}$ as

\begin{align}
\frac{dES}{dt} = &\, -a E_0 \cdot S + (a S + b + c) ES
\end{align}

#### Part a:

Solve $\frac{dES}{dt} = 0$ to find the quasi-equilibrium for $ES$.





*Click here to input answer*

#### Part b:
Use your solution to Part a to write an equation for $\frac{dP}{dt}$ in terms of $S$. When $S\gg E_0$ we see $\frac{dP}{dt}$ becomes constant. What is the maximum rate of change of $P(t)$?

*Click here to input answer*

## Reflection:

What did you think of this lab?

Do you have any suggestions for imprvement[](<- I have left this error in for the discerning reader :D )?

Are there directions or problems you'd like to explore more?